# TOPIC MODELING

In [5]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np





#***************************************************************************************
# 1. Cargamos el corpus de textos
#***************************************************************************************

print("\n\n1. Cargamos el corpus")
trainCorpus = ["Me gustan las vacas",
               "Me gustan los caballos",
               "odio los perros",
               "odio los caballos",
               "me gustan las ranas",
               "me gusta el helado",
               "no quiero comer",
               "los helados son cremosos"]
print (trainCorpus)




#*********************************************************************************************************
# 2. Vectorizamos los textos del corpus (convertimos cada texto en un vector de frecuencias de palabras)
#*********************************************************************************************************
print("\n\n2. Vectorizamos los textos")

puncts = [c for c in string.punctuation]
spanish_stopwords = nltk.corpus.stopwords.words('spanish')+puncts
vectorizer = CountVectorizer(stop_words=spanish_stopwords)

#Transformamos los documentos en una matriz de tf's de documentos.
vectorizer.fit(trainCorpus) #El vectorizador aprende el vocabulario del corpus
print ("\nAtributos:",vectorizer.get_feature_names())
tfMatrix = vectorizer.transform(trainCorpus) #Extraemos las frecuencias de palabras (tf)
print ("\nMatriz tf:\n",tfMatrix.toarray())


#La matriz tf es nuestro dataset, donde:
# - cada fila representa una muestra (un documento del corpus)
# - cada columna representa un atributo (la frecuencia de una palabra en dicho documento)
print("\nnº de muestras: %d, nº de atributos: %d" % tfMatrix.shape)
tf_feature_names = vectorizer.get_feature_names()





#*********************************************************************************************************
# 3. Aplicamos el algoritmo LDA para extraer los topics
#*********************************************************************************************************
topics = 3
lda_model = LatentDirichletAllocation(n_components =topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tfMatrix)
H = lda_model.components_
W = lda_model.transform(tfMatrix)

no_top_words = 3
no_top_documents = 3

def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\nTopic %d:" % (topic_idx))
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            print(" ",feature_names[i])
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(trainCorpus[doc_index])

display_topics(H, W, tf_feature_names, trainCorpus, no_top_words, no_top_documents)



1. Cargamos el corpus
['Me gustan las vacas', 'Me gustan los caballos', 'odio los perros', 'odio los caballos', 'me gustan las ranas', 'me gusta el helado', 'no quiero comer', 'los helados son cremosos']


2. Vectorizamos los textos

Atributos: ['caballos', 'comer', 'cremosos', 'gusta', 'gustan', 'helado', 'helados', 'odio', 'perros', 'quiero', 'ranas', 'vacas']

Matriz tf:
 [[0 0 0 0 1 0 0 0 0 0 0 1]
 [1 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 1 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 0]]

nº de muestras: 8, nº de atributos: 12

Topic 0:
  caballos
  gustan
  cremosos
Me gustan los caballos
los helados son cremosos
me gustan las ranas

Topic 1:
  gustan
  quiero
  comer
no quiero comer
Me gustan las vacas
me gustan las ranas

Topic 2:
  odio
  helado
  gusta
me gusta el helado
odio los perros
odio los caballos
